# Image Classification with Tensorflow (1.x)

This example is built with Tensorflow 1.5 (tensorflow 2.x support for RedisAI is can be implemented using graph freezing. Checkout the [documentation](https://github.com/RedisAI/RedisAI)). We first take a prebuilt model from Tensorflow hub and then push that to RedisAI for production. Towards the end of this example, we also show how the loaded model can be used for inference

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import json
import time
from redisai import Client
from ml2rt import load_model, load_script, save_tensorflow
from skimage import io
import os

## Downloading and save the model
The pretrained model is downloaded from the tensorlfow hub. We then use a dummy input `image` to serialize the graph into the disk

In [3]:
url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/1'
images = tf.placeholder(tf.float32, shape=(1, 224, 224, 3), name='images')
module = hub.Module(url)
logits = module(images)
logits = tf.identity(logits, 'output')

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer()])
    save_tensorflow(sess, 'resnet50.pb', output=['output'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Froze 272 variables.


INFO:tensorflow:Froze 272 variables.


INFO:tensorflow:Converted 272 variables to const ops.


INFO:tensorflow:Converted 272 variables to const ops.


## Setup RedisAI
This tutorial assumes you already have a RedisAI server running. The easiest way to setup one instance is using docker

```
docker run -p 6379:6379 redislabs/redisai:latest-cpu-x64-bionic
```

Take a look at this [quickstart](https://oss.redis.com/redisai/quickstart/) for more details. Here we setup the connection credentials and ping the server to verify we can talk 

In [4]:
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT", 6379))
con = Client(host=REDIS_HOST, port=REDIS_PORT)
con.ping()

True

## Load model
Next step is to load the model we trained above into RedisAI for serving. We are using a convinent package [ml2rt](https://pypi.org/project/ml2rt/) here for loading but it's not a mandatory dependency if you want to keep your `requirements.txt` small. Take a look at the `load_model` function. This will give us a binary blob of the model we have built above. We need to send this to RedisAI and also inform which backend we'd like to use and which device this should run on. We'll set the model on a key so we can reference this key later

Note: If you want to run on GPU, take a look at the above quick start to setup RedisAI on GPU

In [6]:
model = load_model("resnet50.pb")
con.modelstore("tensorflow_model", backend="TF", device="CPU", inputs=['images'], outputs=['output'], data=model)

'OK'

## Load script
Why do you need Script? It's very likely that your deep learning model would have a pre/post processing step, like changing the dimensionality of the input (adding batch dimension) or doing normalizatoin etc. You normally do this from your client code and send the processed data to model server. With script, you can club this into your model serving pipeline. Script is one of the powerful feature of RedisAI. RedisAI Scripts are built on top of [TorchScript](https://pytorch.org/docs/stable/jit.html) and it's recommended to take a look if TorcScript is new to you. Torchscript is a subset of python programming langauge i.e it looks and smells like python but all the python functionalities are not available in torchscript. Now if you are wondering what's the benefit of TorchScript in RedisAI, there are few

- It runs on a highly effecient C++ runtime
- It can pipeline your preprocessing and postprocessing jobs, right where your model and data resides. So no back and forth of huge data blobs between your model server and pre/post processing scripts
- It can run in a single redis pipeline or in RedisAI Dag which makes serving channel implementation smooth
- As in this example, even if your model is in Tensorflow, you can use Script to pipe the worflow

You can load the script from a file (`ml2rt.load_script` does this for you) which is probably your workflow normally since you save the script in a file but here we pass the string into the `scriptstore` method

In [7]:
script = """
def pre_process(tensors: List[Tensor], keys: List[str], args: List[str]):
    image = tensors[0]
    return image.float().div(255).unsqueeze(0)

def post_process(tensors: List[Tensor], keys: List[str], args: List[str]):
    output = tensors[0]
    # tf model has 1001 classes, hence negative 1
    return output.max(1)[1] - 1
"""
con.scriptstore("processing_script", device="CPU", script=script, entry_points=("pre_process", "post_process"))

'OK'

## Load the image and final classes
Here we load the input image and the final classes to find the predicted output

In [8]:
image = io.imread("data/cat.jpg")
class_idx = json.load(open("data/imagenet_classes.json"))

## Run the model serving pipeline
Here we run the serving pipeline one by one and finally fetch the results out. The pipeline is organized into 5 steps

```
Setting Input -> Pre-processing Script -> Running Model -> Post-processing Script -> Fetching Output
```

In [10]:
con.tensorset('image', image)
out4 = con.scriptrun('processing_script', 'pre_process', inputs='image', outputs='processed')
out5 = con.modelrun('tensorflow_model', 'processed', 'model_out')
out6 = con.scriptrun('processing_script', 'post_process', 'model_out', 'final')
final = con.tensorget('final')
print(final[0], class_idx[str(final[0])])

281 tabby, tabby catamount


/Users/hhsecond/asgard/redisai-examples/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated method scriptrun. (Use scriptexecute instead) -- Deprecated since version 1.2.0.
  
/Users/hhsecond/asgard/redisai-examples/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated method modelrun. (Use modelexecute instead) -- Deprecated since version 1.2.0.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hhsecond/asgard/redisai-examples/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated method scriptrun. (Use scriptexecute instead) -- Deprecated since version 1.2.0.
  after removing the cwd from sys.path.
